## Imports and Path Setup

In [ ]:
import sys
import os
import pandas as pd
import ast
import matplotlib.pyplot as plt
import torch

# Add parent directory to path to import models from root
sys.path.append(os.path.abspath('..'))

from models import InstructDeBERTa, BaselineModel
from evaluate import evaluate_model

# Ensure plots display inline
%matplotlib inline

## Load Demo Data

In [ ]:
# Load the demo dataset from the local CSV
data_path = 'demo_data.csv'
print(f"Loading data from {data_path}...")

df = pd.read_csv(data_path)

# Parse stringified lists into Python objects
dataset = []
for _, row in df.iterrows():
    try:
        gt = ast.literal_eval(row['ground_truth'])
        dataset.append({'text': row['text'], 'ground_truth': gt})
    except Exception as e:
        print(f"Skipping row: {e}")

print(f"Successfully loaded {len(dataset)} examples.")
df.head()

## Initialize Models

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running on: {device}")

# Initialize Proposed Model (Beam Size = 1)
print("Loading Instruct-DeBERTa (Beam=1)...")
model_v1 = InstructDeBERTa(device=device, beam_size=1)

# Initialize Proposed Model with Hyperparameter Change (Beam Size = 3)
print("Loading Instruct-DeBERTa (Beam=3)...")
model_v2 = InstructDeBERTa(device=device, beam_size=3)

# Initialize Baseline
print("Loading Baseline (DistilBERT)...")
model_base = BaselineModel(device=device)

## Run Analysis

In [ ]:
print("Evaluating Baseline...")
res_base = evaluate_model(model_base, dataset)

print("Evaluating Instruct-DeBERTa (Beam=1)...")
res_v1 = evaluate_model(model_v1, dataset)

print("Evaluating Instruct-DeBERTa (Beam=3)...")
res_v2 = evaluate_model(model_v2, dataset)

# Compile Results
results = pd.DataFrame([
    {'Model': 'Baseline', **res_base},
    {'Model': 'Instruct-DeBERTa (B=1)', **res_v1},
    {'Model': 'Instruct-DeBERTa (B=3)', **res_v2}
])

results = results.replace({0.0: 0}) # Clean up for visualization
display(results)

## Generate Graphical Outputs

In [ ]:
# Visualization
metrics = ['ATE_F1', 'ASC_Accuracy', 'ASC_F1']
models = results['Model'].tolist()

x = range(len(metrics))
width = 0.25

fig, ax = plt.subplots(figsize=(10, 6))

colors = ['#e74c3c', '#3498db', '#2ecc71']

for i, model in enumerate(models):
    vals = results.loc[i, metrics].values
    offset = (i - len(models)/2) * width + width/2
    rects = ax.bar([p + offset for p in x], vals, width, label=model, color=colors[i])
    ax.bar_label(rects, padding=3, fmt='%.2f')

ax.set_ylabel('Scores')
ax.set_title('Performance on Laptop Domain (Demo Data)')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend(loc='lower right')
ax.set_ylim(0, 1.2)

plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.show()

## Interactive Prediction

In [ ]:
# Try your own sentence here!
text = "The processor speed is fantastic, but it overheats quickly."

print(f"Input: {text}\n")
print(f"Baseline Prediction: {model_base.predict_sentiment(text)}")
print(f"Instruct-DeBERTa Prediction: {model_v1.predict(text)}")